In [1]:
#computes sliding window classification accuracy for orofacial movements, single phonemes, and single words for all 4
#arrays (Fig 1e)
baseDir = '/oak/stanford/groups/shenoy/fwillett/speechPaperRelease_final/'

In [2]:
import scipy.io

phonemesDat = scipy.io.loadmat(baseDir+'tuningTasks/t12.2022.04.26_phonemes.mat')
orofacialDat = scipy.io.loadmat(baseDir+'tuningTasks/t12.2022.04.21_orofacial.mat')
fiftyWordDat = scipy.io.loadmat(baseDir+'tuningTasks/t12.2022.05.03_fiftyWordSet.mat')
allDatasets = [orofacialDat, phonemesDat, fiftyWordDat]

In [3]:
#mean-subtract within block
import numpy as np
def meanSubtract(dat):
    dat['feat'] = dat['tx2'].astype(np.float32)    
    blockList = np.squeeze(np.unique(dat['blockNum']))
    for b in blockList:
        loopIdx = np.squeeze(dat['blockNum']==b)
        dat['feat'][loopIdx,:] -= np.mean(dat['feat'][loopIdx,:],axis=0,keepdims=True)
    return dat

In [4]:
#mean subtraction
from analysis import unscrambleChans, triggeredAvg
for x in range(len(allDatasets)):
    allDatasets[x] = meanSubtract(allDatasets[x])
    allDatasets[x]['feat'][:,0:128] = unscrambleChans(allDatasets[x]['feat'][:,0:128])
    allDatasets[x]['feat'][:,128:] = unscrambleChans(allDatasets[x]['feat'][:,128:])

2023-06-14 13:58:58.245981: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 13:58:58.448383: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-14 13:59:04.434291: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /share/software/user/open/gcc/10.1.0/lib64:/share/software/user/open/gcc/10.1.0/lib/gcc/x86_64-pc-linux-gnu:/share/software/user/open/gcc/10.1.0/lib:/share/software/user/open/cudnn/8.6.0.163/lib:/usr/lib64/nvidia

In [5]:
dataset_trials = []
for x in range(len(allDatasets)):
    fAvg, fCI, trials_go = triggeredAvg(allDatasets[x]['feat'].astype(np.float32), 
                             allDatasets[x]['goTrialEpochs'][:,0],
                             np.squeeze(allDatasets[x]['trialCues']), [-25,125], smoothSD=4)

    fAvg, fCI, trials_delay = triggeredAvg(allDatasets[x]['feat'].astype(np.float32), 
                             allDatasets[x]['delayTrialEpochs'][:,0],
                             np.squeeze(allDatasets[x]['trialCues']), [0,150], smoothSD=4)
    
    dataset_trials.append([trials_go, trials_delay])

In [6]:
fiftyWordCuesToUse = np.arange(1,51).astype(np.int32)
phonemeCuesToUse = np.array([0,1,3,4,5,6,7,8,9,10,11,12,14,15,16,17,19,20,21,22,23,24,
                      26,27,28,29,30,31,32,33,34,35,36,37,38,39]).astype(np.int32)
oroCuesToUse = np.array([0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
                         21,22,23,24,25,26,27,28,29,30,31,32,33]).astype(np.int32)
allCues = [oroCuesToUse, phonemeCuesToUse, fiftyWordCuesToUse]

In [ ]:
from analysis import gnb_loo, bootCI
#get classification accuracy for each window & task (plus 95% CIs computed via bootstrap), this may take a while
dataset_results = []
for datasetIdx in range(len(allDatasets)):
    trialSets = [dataset_trials[datasetIdx][0], dataset_trials[datasetIdx][1]]
    all_results = []

    for trialSetIdx in range(len(trialSets)):
        trials_subset = []
        for t in range(len(allCues[datasetIdx])):
            trials_subset.append(trialSets[trialSetIdx][allCues[datasetIdx][t]])

        chanSets = [np.arange(0,64).astype(np.int32), np.arange(64,128).astype(np.int32),
                   np.arange(128,192).astype(np.int32), np.arange(192,256).astype(np.int32)]

        all_vals = []
        for chanSetIdx in range(len(chanSets)):
            mns = []
            cis = []
            for n in range(125):   
                y_pred, unroll_y = gnb_loo(trials_subset, [n, n+5], chanSets[chanSetIdx])
                mns.append(np.mean(y_pred==unroll_y))
                cis.append(bootCI(y_pred, unroll_y))
                print(np.mean(y_pred==unroll_y))

            all_vals.append(np.concatenate([np.array(mns)[:,np.newaxis], np.array(cis)], axis=1))

        all_results.append(all_vals)
    
    dataset_results.append(all_results)
        

0.1244309559939302
0.1456752655538695
0.13808801213960548
0.12898330804248861
0.13201820940819423
0.1456752655538695
0.1471927162367223
0.15629742033383914
0.1471927162367223
0.1411229135053111
0.13960546282245828
0.13808801213960548
0.11077389984825493
0.12291350531107739
0.11229135053110774
0.11836115326251896
0.13657056145675264
0.11987860394537178
0.1456752655538695
0.15326251896813353
0.14871016691957512
0.15174506828528073
0.1471927162367223
0.17147192716236723
0.165402124430956
0.165402124430956
0.17147192716236723
0.1669195751138088
0.13960546282245828
0.1456752655538695
0.1638846737481032
0.13353566009104703
0.15781487101669195
0.1669195751138088
0.16995447647951442
0.19575113808801214
0.18816388467374812
0.19878603945371776
0.18512898330804248
0.22154779969650987
0.2276176024279211
0.24279210925644917
0.26555386949924126
0.27921092564491656
0.27617602427921095
0.291350531107739
0.291350531107739
0.2852807283763278
0.26100151745068284
0.26858877086494687
0.27769347496206376
0.

In [ ]:
#get average audio volume envelope
fAvg_phonemes, fCI_phonemes, trials_phonemes = triggeredAvg(phonemesDat['audioEnvelope'].astype(np.float32), 
                         phonemesDat['goTrialEpochs'][:,0],
                         np.squeeze(phonemesDat['trialCues']), [-25,100], smoothSD=0)
audioPhonemes = np.mean(fAvg_phonemes[:,:,0],axis=0)

fAvg_fiftyWord, fCI_fiftyWord, trials_fiftyWord = triggeredAvg(fiftyWordDat['audioEnvelope'].astype(np.float32), 
                         fiftyWordDat['goTrialEpochs'][:,0],
                         np.squeeze(fiftyWordDat['trialCues']), [-25,100], smoothSD=0)
audioFiftyWord = np.mean(fAvg_fiftyWord[:,:,0],axis=0)

In [ ]:
#plot sliding window classification for each array and task
import matplotlib.pyplot as plt
from analysis import plotPreamble
plotPreamble()

chance = [1/33, 1/39, 1/50]
colors = ['firebrick','tomato','royalblue','blueviolet']
taskTitles = ['Orofacial Movements','Single Phonemes','Words']
plt.figure(dpi=300,figsize=(6,1))
for taskIdx in range(len(chance)):
    plt.subplot(1,3,taskIdx+1)
    
    goDat = dat['results'][taskIdx,0,:,:,:]*100
    delayDat = dat['results'][taskIdx,1,:,0:100,:]*100
            
    lHandles = []
    for arrIdx in range(4):
        lHandle = plt.plot(np.arange(0,100,5)*0.02, delayDat[arrIdx,0:-1:5,0],color=colors[arrIdx])
        lHandles.append(lHandle[0])
        
        plt.fill_between(np.arange(0,100,5)*0.02, 
             delayDat[arrIdx,0:-1:5,1], 
             delayDat[arrIdx,0:-1:5,2],alpha=0.3,color=colors[arrIdx])
            
        plt.plot(150*0.02 + np.arange(-25,100,5)*0.02, goDat[arrIdx,0:-1:5,0],color=colors[arrIdx])
        plt.fill_between(150*0.02 + np.arange(-25,100,5)*0.02, 
             goDat[arrIdx,0:-1:5,1], 
             goDat[arrIdx,0:-1:5,2],alpha=0.3,color=colors[arrIdx])
        
    plt.xlabel('Time (s)')
    if taskIdx==0:
        plt.ylabel('Classification\nAccuracy (%)')
    plt.title(taskTitles[taskIdx])
    plt.ylim([0,65])
    if taskIdx==2:
        plt.gca().legend(lHandles, ['6v dorsal','6v ventral','44 dorsal','44 ventral'],frameon=False)
        
    if taskIdx==1:
        speechAudioLine = audioPhonemes.copy()
        speechAudioLine = speechAudioLine - min(speechAudioLine)
        speechAudioLine = speechAudioLine / max(speechAudioLine)
        plt.plot(150*0.02 + np.arange(-25,100)*0.02, speechAudioLine*65,color='darkgray',alpha=0.5,linewidth=2)
    
    if taskIdx==2:
        speechAudioLine = audioFiftyWord.copy()
        speechAudioLine = speechAudioLine - min(speechAudioLine)
        speechAudioLine = speechAudioLine / max(speechAudioLine)
        plt.plot(150*0.02 + np.arange(-25,100)*0.02, speechAudioLine*65,color='darkgray',alpha=0.5,linewidth=2)
            
    plt.xticks(ticks=[0,1,2,3,4,5], labels=['0','1','2','0','1','2'])
    plt.plot([0,0],plt.gca().get_ylim(),'--k',linewidth=0.75)
    plt.plot([3,3],plt.gca().get_ylim(),'--k',linewidth=0.75)
    plt.plot(plt.gca().get_xlim(),[chance[taskIdx]*100, chance[taskIdx]*100],'--k',linewidth=0.75)
    
plt.show()